# Pair Programming Métricas Regresión Lineal
## Ana C y Ana G

En el pair programming anterior creastéis vuestro primer modelo de machine learning usando la regresion Lineal. 

Es el momento, que con vuestros datos evaluéis si es bueno haciendo predicciones. 

Los objetivo de este pairprogramming son:

- Calculéis las métricas para vuestro modelo
- Discutid los resultados de las métricas y extraed conclusiones
- Guardad los resultados de las métricas en un csv para usarlo más adelante.

In [27]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Primas bajas

In [28]:
df=pd.read_csv('../data/08-primas_categorizadas_normalizadas.csv',index_col=0)#primas bajas
df.head()

,age,sex,bmi,children,smoker,region,charges_Sklearn,fumadores,hombres,northeast,northwest,southeast,southwest
0,-1.387916,female,-0.382685,-0.885991,yes,southwest,0.836552,1,0,0.0,0.0,0.0,1.0
1,-1.459488,male,0.601634,-0.069805,no,southeast,0.032037,0,1,0.0,0.0,1.0,0.0
2,-0.743769,male,0.472516,1.562566,no,southeast,0.176597,0,1,0.0,0.0,1.0,0.0
3,-0.457482,male,-0.218352,-0.885991,no,northwest,0.145677,0,1,0.0,1.0,0.0,0.0
4,-0.529054,female,-0.744888,-0.885991,no,southeast,0.139827,0,0,0.0,0.0,1.0,0.0


In [29]:
X = df.drop(['charges_Sklearn','bmi','smoker','region','sex','northeast','northwest','southeast','southwest'], axis = 1)#predictoras
y = df["charges_Sklearn"]# respuestas

In [30]:
X.head(2)

,age,children,fumadores,hombres
0,-1.387916,-0.885991,1,0
1,-1.459488,-0.069805,0,1


In [31]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [32]:
lr = LinearRegression(n_jobs=-1)

In [33]:
lr.fit(x_train, y_train)

LinearRegression(n_jobs=-1)

In [34]:
y_predict_train = lr.predict(x_train) # hacemos las predicciones
y_predict_test = lr.predict(x_test)

In [35]:
train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_predict_train, 'Set': ['Train']*len(y_train)})
test_df  = pd.DataFrame({'Real': y_test,  'Predicted': y_predict_test,  'Set': ['Test']*len(y_test)})
resultados = pd.concat([train_df,test_df], axis = 0)
resultados.head()

,Real,Predicted,Set
72,0.483175,0.495159,Train
903,0.109029,0.144614,Train
44,0.159836,0.197193,Train
309,0.820715,0.055763,Train
788,0.345104,0.403968,Train


In [36]:
resultados['residuos'] = resultados['Real'] - resultados['Predicted']
resultados.head()

,Real,Predicted,Set,residuos
72,0.483175,0.495159,Train,-0.011984
903,0.109029,0.144614,Train,-0.035585
44,0.159836,0.197193,Train,-0.037357
309,0.820715,0.055763,Train,0.764952
788,0.345104,0.403968,Train,-0.058864


In [37]:
resultados_metricas = {'MAE': [mean_absolute_error(y_test, y_predict_test), mean_absolute_error(y_train, y_predict_train)],
                'MSE': [mean_squared_error(y_test, y_predict_test), mean_squared_error(y_train, y_predict_train)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_predict_test)), np.sqrt(mean_squared_error(y_train, y_predict_train))],
                'R2':  [r2_score(y_test, y_predict_test), r2_score(y_train, y_predict_train)],
                 "set": ["test", "train"], 
                 "modelo": ["Linear Regresion", "LinearRegression"]}

df_resultados = pd.DataFrame(resultados_metricas)

df_resultados

,MAE,MSE,RMSE,R2,set,modelo
0,0.061607,0.016929,0.130112,0.733553,test,Linear Regresion
1,0.056192,0.013757,0.117289,0.778495,train,LinearRegression


---

Resultado:

- Mas del 70% de la variación de nuestra **variable respuesta ** es explicado por nuestras variables predictoras,
-  y los valor de MAE y MSE estan muy cerca del cero, segun esto seria un buen modelo predictivo, 
- pero MSE y MAE de el grupo **train**, son menores que las del grupo **test**, 
- al contrario que ocurre con los valores de R2.
- Esto podria indicar, **overfitting**, pero en nuestro caso, las diferencias son bajas, más bien, quiere decir, que tenemos un modelo bastante bueno.

---

# Primas altas

In [38]:
df2=pd.read_csv('../data/09-primas-altas.csv',index_col=0)# primas altas sin normalizar
df2.head()

,age,sex,bmi,children,smoker,region,charges,fumadores,sex_female,sex_male
3,33,male,22.705,0,no,northwest,21984.47061,0,0,1
9,60,female,25.840,0,no,northwest,28923.13692,0,1,0
11,62,female,26.290,0,yes,southeast,27808.72510,1,1,0
14,27,male,42.130,0,yes,southeast,39611.75770,1,0,1
19,30,male,35.300,0,yes,southwest,36837.46700,1,0,1


In [39]:
numericas2= df2.select_dtypes(np.number)
numericas2

,age,bmi,children,charges,fumadores,sex_female,sex_male
3,33,22.705,0,21984.47061,0,0,1
9,60,25.840,0,28923.13692,0,1,0
11,62,26.290,0,27808.72510,1,1,0
14,27,42.130,0,39611.75770,1,0,1
19,30,35.300,0,36837.46700,1,0,1
...,...,...,...,...,...,...,...
1313,19,34.700,2,36397.57600,1,1,0
1321,62,26.695,0,28101.33305,1,0,1
1323,42,40.370,2,43896.37630,1,1,0
1328,23,24.225,2,22395.74424,0,1,0


In [40]:
X = numericas2.drop(['bmi','charges'],axis=1)#predictoras
y = df2["charges"]# respuestas

In [41]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [42]:
lr = LinearRegression(n_jobs=-1)

In [43]:
lr.fit(x_train, y_train)

LinearRegression(n_jobs=-1)

In [44]:
y_predict_train = lr.predict(x_train) 
y_predict_test = lr.predict(x_test)

In [45]:
train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_predict_train, 'Set': ['Train']*len(y_train)})
test_df  = pd.DataFrame({'Real': y_test,  'Predicted': y_predict_test,  'Set': ['Test']*len(y_test)})
resultados = pd.concat([train_df,test_df], axis = 0)
resultados.head()

,Real,Predicted,Set
140,27375.90478,25522.202120,Train
947,39047.28500,35764.982927,Train
682,40103.89000,36219.153199,Train
1007,24915.22085,36994.706213,Train
158,36950.25670,35042.993703,Train


In [46]:
resultados['residuos'] = resultados['Real'] - resultados['Predicted']
resultados.head()

,Real,Predicted,Set,residuos
140,27375.90478,25522.202120,Train,1853.702660
947,39047.28500,35764.982927,Train,3282.302073
682,40103.89000,36219.153199,Train,3884.736801
1007,24915.22085,36994.706213,Train,-12079.485363
158,36950.25670,35042.993703,Train,1907.262997


In [47]:
resultados_metricas2 = {'MAE': [mean_absolute_error(y_test, y_predict_test), mean_absolute_error(y_train, y_predict_train)],
                'MSE': [mean_squared_error(y_test, y_predict_test), mean_squared_error(y_train, y_predict_train)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_predict_test)), np.sqrt(mean_squared_error(y_train, y_predict_train))],
                'R2':  [r2_score(y_test, y_predict_test), r2_score(y_train, y_predict_train)],
                 "set": ["test", "train"], 
                 "modelo": ["Linear Regresion", "LinearRegression"]}

df_resultados2 = pd.DataFrame(resultados_metricas2)

df_resultados2

,MAE,MSE,RMSE,R2,set,modelo
0,6753.719242,8.026782e+07,8959.230964,0.222684,test,Linear Regresion
1,6612.818781,6.994785e+07,8363.483322,0.215961,train,LinearRegression


---

Resultados:

- En este modelo vemso que tiene un bajo valor predictivo, el valor de R no supera el 25% de predicción y los valores de MAE y MSE son muy elevados. 
- Hay que tener en cuenta que estos valores no estan normalizados.
- Tendriamos ue aumentar el numero de variables predictoras, aumentar la **n** o revisar la partición del dataset inicial.

---

# Resultados para las dos particiones (primas bajas y primas altas)

In [48]:
df_resultados.head()

,MAE,MSE,RMSE,R2,set,modelo
0,0.061607,0.016929,0.130112,0.733553,test,Linear Regresion
1,0.056192,0.013757,0.117289,0.778495,train,LinearRegression


In [49]:
df_resultados2.head()

,MAE,MSE,RMSE,R2,set,modelo
0,6753.719242,8.026782e+07,8959.230964,0.222684,test,Linear Regresion
1,6612.818781,6.994785e+07,8363.483322,0.215961,train,LinearRegression


---

In [51]:
df_resultados.to_csv("../data/10-resultados_metricas.csv")

In [50]:
df_resultados2.to_csv("../data/10-resultados_metricas_altas.csv")